# --- Day 8: Playground ---

Their plan is to connect the junction boxes with long strings of lights. Most of the junction boxes don't provide electricity; however, when two junction boxes are connected by a string of lights, electricity can pass between those two junction boxes.

The Elves are trying to figure out which junction boxes to connect so that electricity can reach every junction box. They even have a list of all of the junction boxes' positions in 3D space (your puzzle input).

For example:
```
162,817,812
57,618,57
906,360,560
592,479,940
352,342,300
466,668,158
542,29,236
431,825,988
739,650,466
52,470,668
216,146,977
819,987,18
117,168,530
805,96,715
346,949,466
970,615,88
941,993,340
862,61,35
984,92,344
425,690,689
```
This list describes the position of 20 junction boxes, one per line. Each position is given as X,Y,Z coordinates. So, the first junction box in the list is at X=162, Y=817, Z=812.

To save on string lights, the Elves would like to focus on connecting pairs of junction boxes that are as close together as possible according to straight-line distance. In this example, the two junction boxes which are closest together are 162,817,812 and 425,690,689.

By connecting these two junction boxes together, because electricity can flow between them, they become part of the same circuit. After connecting them, there is a single circuit which contains two junction boxes, and the remaining 18 junction boxes remain in their own individual circuits.

Now, the two junction boxes which are closest together but aren't already directly connected are 162,817,812 and 431,825,988. After connecting them, since 162,817,812 is already connected to another junction box, there is now a single circuit which contains three junction boxes and an additional 17 circuits which contain one junction box each.

The next two junction boxes to connect are 906,360,560 and 805,96,715. After connecting them, there is a circuit containing 3 junction boxes, a circuit containing 2 junction boxes, and 15 circuits which contain one junction box each.

The next two junction boxes are 431,825,988 and 425,690,689. Because these two junction boxes were already in the same circuit, nothing happens!

This process continues for a while, and the Elves are concerned that they don't have enough extension cables for all these circuits. They would like to know how big the circuits will be.

After making the ten shortest connections, there are 11 circuits: one circuit which contains 5 junction boxes, one circuit which contains 4 junction boxes, two circuits which contain 2 junction boxes each, and seven circuits which each contain a single junction box. Multiplying together the sizes of the three largest circuits (5, 4, and one of the circuits of size 2) produces 40.

Your list contains many junction boxes; connect together the 1000 pairs of junction boxes which are closest together. Afterward, what do you get if you multiply together the sizes of the three largest circuits?

In [ ]:
from fastcore.utils import L
from aocd import get_data

inp = get_data(day=8, year=2025)
samp = '''162,817,812
57,618,57
906,360,560
592,479,940
352,342,300
466,668,158
542,29,236
431,825,988
739,650,466
52,470,668
216,146,977
819,987,18
117,168,530
805,96,715
346,949,466
970,615,88
941,993,340
862,61,35
984,92,344
425,690,689'''
samp

Summarize the problem for me.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠

This is a classic **Minimum Spanning Tree** style problem with a twist. Here's the gist:

**Setup:** You have junction boxes at 3D coordinates (x, y, z). You need to connect them with "string lights."

**Process:**
1. Find the two closest junction boxes (by Euclidean distance) and connect them
2. Repeat: always connect the next closest pair that aren't **already directly connected**
3. Connections form "circuits" — if A connects to B, and B connects to C, they're all in one circuit (this is the **Union-Find** pattern!)

**Key detail:** Even if two boxes are already in the same circuit (indirectly connected), you still make the connection if they're the closest unconnected pair — it just doesn't change the circuit structure.

**Task:** After making the **1000 shortest connections**, multiply the sizes of the **three largest circuits**.

The sample shows that after 10 connections on 20 boxes, you get circuits of sizes 5, 4, 2, 2, 1, 1, 1, 1, 1, 1, 1 → answer is 5 × 4 × 2 = 40.

**Algorithm hint:** You'll want to:
1. Compute all pairwise distances
2. Sort pairs by distance
3. Use Union-Find to track which boxes are in which circuit
4. Process the 1000 shortest connections

Does that make sense? Ready to start coding it?

In [ ]:
xs = samp.splitlines()
xs

In [ ]:
x = xs[0]
x

In [ ]:
b = tuple(L(x.split(',')).map(int))
b

In [ ]:
def parse_box(x):
    return tuple(L(x.split(',')).map(int))
parse_box(x)

In [ ]:
def parse(text):
    return L(text.splitlines()).map(parse_box)
bs = parse(samp)
bs

In [ ]:
from math import *

dist(b1, b2)

Where we are at?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You've successfully:
- Parsed the sample input into a list of 3D coordinate tuples
- Created a `parse_box()` function and a `parse()` function to handle the conversion
- Started exploring distance calculations (I see you've used `dist(b1, b2)` which gave ~787.8)

The next step would be to generate all pairs of junction boxes with their distances, then sort them to find the shortest connections. You'll also need a Union-Find structure to track which boxes are in the same circuit as you make connections.

Would you like to work on:
1. Computing all pairwise distances?
2. Setting up the Union-Find data structure?
3. Something else?

What feels like the natural next step to you?

What would be an ideal representation for the pairwise distances? I'm thinking about a list or a matrix.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good question! Let's think about what you need to *do* with the pairwise distances:

You need to sort all pairs by distance and process them in order. You don't need to look up "what's the distance between box i and box j?" — you just need "what are all the pairs, ordered by distance?"

Given that, which representation would work better for your use case?

I think list will do fine.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Perfect! A list of `(distance, box_i, box_j)` tuples would work well — you can sort it once and then iterate through it.

How would you generate all the pairs? Do you want to sketch out the approach, or would you like a hint about avoiding duplicates (like counting both (i,j) and (j,i))?

In [ ]:
ds = []
for i in range(len(bs)):
    for j in range(i + 1, len(bs)):
        d = dist(bs[i], bs[j])
        ds.append((d, i, j))
ds

In [ ]:
def dists(bs):
    ds = []
    for i in range(len(bs)):
        for j in range(i + 1, len(bs)):
            d = dist(bs[i], bs[j])
            ds.append((d, i, j))
    return ds
dists(bs)

How can I make it more functional

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You could use a list comprehension instead of building the list with `append`. Something like:

```python
[(dist(bs[i], bs[j]), i, j) for i in range(len(bs)) for j in range(i+1, len(bs))]
```

This creates the same list in a more functional style. Want to try refactoring your `dists` function with this approach?

What would be a better name for the dists function?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good question! `dists` is a bit vague. Some options:

- `pairwise_distances` — clear and descriptive
- `all_pairs_with_distances` — very explicit
- `compute_distances` — action-oriented
- `get_pairs` — shorter, focuses on what you're getting

What feels right to you? Or do you have another idea?

In [ ]:
def pairwise_dist(bs):
    return [(dist(bs[i], bs[j]), i, j) for i in range(len(bs)) for j in range(i+1, len(bs))]
ds = pairwise_dist(bs)
ds

In [ ]:
ds = sorted(ds)
ds

What's next?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Now you need the Union-Find data structure to track which boxes belong to which circuit as you make connections.

Have you implemented Union-Find before, or would you like some guidance on how to approach it? The key operations you'll need are:
- `find(x)` — which circuit does box x belong to?
- `union(x, y)` — merge the circuits containing x and y

What would you like to do?

I'm not sure which data structure to use. Can you help me?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Sure! For Union-Find, you have a few options:

1. **Dictionary-based** — map each box index to its "parent" or "root" circuit identifier
2. **List-based** — use a list where `parent[i]` tells you the parent of box `i`
3. **Use a library** — like `networkx` which has union-find built in

For this problem, a simple list or dictionary works well. You'd track:
- Which circuit each box belongs to (initially, each box is its own circuit)
- Optionally, the size of each circuit (to find the three largest at the end)

Which approach sounds most comfortable to you? Or would you like me to sketch out a small example of how one of these would work?

Using a list should work, I guess.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Great! With a list-based approach, you'd have something like:

- `parent = list(range(len(bs)))` — initially, each box is its own parent (circuit of size 1)

Then you need a `find(x)` function that follows the parent chain to find the root, and a `union(x, y)` function that connects two circuits.

Want to try writing the `find` function first? It should keep following `parent[x]` until it finds a box that is its own parent (the root).

Why do I need a parent chain? Wouldn't simply having a component as a value work?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You're absolutely right! You could use a simpler approach where each box just stores which component/circuit it belongs to, without the parent chain.

So something like `component = list(range(len(bs)))` where `component[i]` is the circuit ID for box `i`.

Then when you union two boxes, you'd need to update all boxes in one circuit to have the new circuit ID. Does that approach make sense to you?

The parent chain approach (with path compression) is more efficient for large datasets, but for this problem size, your simpler approach should work fine. Which would you prefer to implement?

In [ ]:
component = list(range(len(bs)))
component

In [ ]:
def union(x, y, component):
    component = component.copy()
    z = min(component[x], component[y])
    for i in range(len(component)):
        component[i] = z if component[i] == x or component[i] == y else component[i]
    return component
union(0, 5, component)

Can I write the union function in a more functional way?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Yes! You can use a list comprehension:

```python
def union(x, y, component):
    cx, cy = component[x], component[y]
    z = min(cx, cy)
    return [z if c == cx or c == cy else c for c in component]
```

Note: I also fixed a bug — you need to check against `component[x]` and `component[y]` (the circuit IDs), not `x` and `y` (the box indices).

In [ ]:
def union(x, y, component):
    cx, cy = component[x], component[y]
    z = min(cx, cy)
    return [z if c == cx or c == cy else c for c in component]
union(0, 5, component)

What do we need next?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Now you need to iterate through the sorted distances, making connections. For each pair, union them and keep track of how many connections you've made.

After making the required number of connections (10 for the sample, 1000 for the real input), you'll need to count the circuit sizes and multiply the three largest.

Want to start with the loop that processes the sorted distances?

In [ ]:
ds[:5]

In [ ]:
from functools import *

In [ ]:
reduce?

In [ ]:
def reduce_fn(component, x):
    _, i, j = x
    return union(i, j, component)
reduce_fn(component, ds[0])

In [ ]:
reduce(reduce_fn , ds[:10], component)

How can I easily count values?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You have a few good options:

1. **`Counter` from `collections`** — `Counter(component)` gives you a dict-like object with counts
2. **Manual with a dict** — iterate and build counts yourself
3. **`pandas.Series.value_counts()`** — if you're comfortable with pandas

`Counter` is probably the most straightforward for this. Want to try it?

In [ ]:
from collections import *

In [ ]:
x = reduce(reduce_fn , ds[:10], component)
x

In [ ]:
Counter(x)

In [ ]:
len(Counter(x))

In [ ]:
Counter(x).most_common(3)

In [ ]:
L(Counter(x).most_common(3)).map(lambda t: t[1])

In [ ]:
L(Counter(x).most_common(3)).map(lambda t: t[1]).product()

Please wrap all this to a function for me.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Sure! Here's a function that wraps it all together:

```python
def solve(text, n_connections):
    boxes = parse(text)
    distances = sorted(pairwise_dist(boxes))
    component = list(range(len(boxes)))
    final = reduce(reduce_fn, distances[:n_connections], component)
    return L(Counter(final).most_common(3)).map(lambda t: t[1]).product()
```

Then you can call it with `solve(samp, 10)` for the sample or `solve(inp, 1000)` for the real input.

In [ ]:
def solve(text, n_connections):
    boxes = parse(text)
    distances = sorted(pairwise_dist(boxes))
    component = list(range(len(boxes)))
    final = reduce(reduce_fn, distances[:n_connections], component)
    return L(Counter(final).most_common(3)).map(lambda t: t[1]).product()
solve(samp, 10)

In [ ]:
solve(inp, 1000)

# --- Part Two ---
The Elves were right; they definitely don't have enough extension cables. You'll need to keep connecting junction boxes together until they're all in one large circuit.

Continuing the above example, the first connection which causes all of the junction boxes to form a single circuit is between the junction boxes at 216,146,977 and 117,168,530. The Elves need to know how far those junction boxes are from the wall so they can pick the right extension cable; multiplying the X coordinates of those two junction boxes (216 and 117) produces 25272.

Continue connecting the closest unconnected pairs of junction boxes together until they're all in the same circuit. What do you get if you multiply together the X coordinates of the last two junction boxes you need to connect?

Summarize the part 2 for me.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Part 2 asks you to keep connecting junction boxes until they're **all in one circuit** (not just 1000 connections).

The answer is: find the **last connection** that merges everything into one circuit, then multiply the **X coordinates** of those two junction boxes.

In the example, boxes at `216,146,977` and `117,168,530` are the final pair → answer is `216 × 117 = 25272`.

How would you modify your solution to find when all boxes are in one circuit, and identify that final pair?

I think I need to replace the reduce function with a while loop and iterate until there are just 2 components and then stop.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good thinking! Though actually you want to stop when there's just **1 component** (all boxes connected), not 2.

Your approach makes sense: loop through the sorted distances, union each pair, and check after each connection if `len(Counter(component)) == 1`. When it is, that's your answer pair.

Want to try coding that up?

Is there an functional alternative to the while loop?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Yes! You could use `itertools.accumulate` to generate all intermediate component states as you process connections, then find the first one where there's only 1 component.

Something like:
- Use `accumulate(distances, reduce_fn, initial=component)` to get all states
- Find the first state where `len(Counter(state)) == 1`
- Track the index to know which distance pair caused it

Want to explore this approach, or would you prefer to stick with the while loop?

But can I stop the iteration early?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good question! `accumulate` will process the entire sequence, which isn't ideal if you want to stop early for efficiency.

For early stopping with a functional style, you could:
1. Use `itertools.takewhile` or `dropwhile` with `accumulate`
2. Write a generator function that yields states and breaks when done
3. Just use a regular loop (sometimes the clearest option!)

The loop is probably most straightforward here since you need to track both the state and which pair caused the final merge. Does that make sense, or do you want to try one of the functional approaches?

In [ ]:
c = component.copy()

for i in range(len(ds)):
    c = reduce_fn(c, ds[i])
    if len(Counter(c)) == 1: break
_, b_i1, b_i2 = ds[i]
bs[b_i1], bs[b_i2]

In [ ]:
x1, _, _ = bs[b_i1]
x2, _, _ = bs[b_i2]
x1 * x2

Wrap this into a function for me.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

```python
def solve2(text):
    boxes = parse(text)
    distances = sorted(pairwise_dist(boxes))
    c = list(range(len(boxes)))
    
    for i in range(len(distances)):
        c = reduce_fn(c, distances[i])
        if len(Counter(c)) == 1: break
    
    _, b_i1, b_i2 = distances[i]
    x1, _, _ = boxes[b_i1]
    x2, _, _ = boxes[b_i2]
    return x1 * x2
```

Try calling it with `solve2(samp)` to verify it gives 25272, then with `solve2(inp)` for your answer!

In [ ]:
def solve2(text):
    boxes = parse(text)
    distances = sorted(pairwise_dist(boxes))
    c = list(range(len(boxes)))
    
    for i in range(len(distances)):
        c = reduce_fn(c, distances[i])
        if len(Counter(c)) == 1: break
    
    _, b_i1, b_i2 = distances[i]
    x1, _, _ = boxes[b_i1]
    x2, _, _ = boxes[b_i2]
    return x1 * x2
solve2(samp)

In [ ]:
solve2(inp)